# Проект классификации комментариев c BERT


Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

<font color='green'>Цель</font>

Выбрать оптимальную модель для предсказания риска ДТП.

<font color='green'>Задачи</font>

- исследовать данные;
- подготовить данные;
- построить модель классификации комментариев со значением метрики качества *F1* не меньше 0.75;


<font color='green'>Файлы</font>

- `toxic_comments.csv` 


<font color='green'>Признаки</font>

- `text` — текст комментария

<font color='green'>Целевой признак</font>

- `toxic` 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-анализ" data-toc-modified-id="Загрузка-и-анализ-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и анализ</a></span></li><li><span><a href="#Обработка-текста" data-toc-modified-id="Обработка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обработка текста</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#TfidfVectorizer" data-toc-modified-id="TfidfVectorizer-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>TfidfVectorizer</a></span></li><li><span><a href="#BertModel" data-toc-modified-id="BertModel-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>BertModel</a></span></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></div>

## Подготовка

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import  f1_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline

import pymorphy2
from string import punctuation
import re
import torch
from torch import cuda
import transformers 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import spacy

import os
import time
import pickle
from tqdm import notebook
import json
import gc

### Загрузка и анализ

In [2]:
pth1 = '/ML/datasets/toxic_comments.csv'
pth2 = 'toxic_comments.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1, index_col=[0], parse_dates=[0])
elif os.path.exists(pth2):
    data = pd.read_csv(pth2, index_col=[0], parse_dates=[0])
else:
    print('Something is wrong')
    
# для BERT    
model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
data.isnull().sum()

text     0
toxic    0
dtype: int64

In [5]:
data.duplicated().sum() 

0

In [6]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

### Обработка текста

In [7]:
stop_words = stopwords.words('english')

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

In [8]:
def preprocess_text_bag(doc):
    
    text = ' '.join([i for i in doc.lower().split() if i not in stop_words]) # строчные буквы + без стоп-слов
    text = re.sub(r'[^a-zA-Z\'\-]', ' ', text) 
    text = re.sub(r'\s+', ' ', text) # только одинарные пробелы
    text = text.strip() # без пробел в начале/конце
    return text

def preprocess_text_bert(doc):
    
    text = re.sub(r'[^a-zA-Z\'\-]', ' ', doc) 
    text = re.sub(r'\s+', ' ', text) # только одинарные пробелы
    text = text.strip() # без пробел в начале/конце
    
    return text

def lemma_text(doc):
    text = nlp(doc)
    text = ' '.join([token.lemma_ for token in text])
    return text

def tokenize_bert(doc): 
    text = tokenizer.encode(doc, add_special_tokens=True, truncation=True, max_length=512) 
    return text

In [9]:
data['lemma'] = [preprocess_text_bag(t) for t in data['text']]
data['lemma_BERT'] = [preprocess_text_bert(t) for t in data['text']]
data['len'] = [len(t) for t in data['lemma_BERT']]

In [10]:
%%time
lemma = [lemma_text(t) for t in data['lemma']]

CPU times: total: 11min 21s
Wall time: 11min 22s


In [11]:
%%time
lemma_bert = [tokenize_bert(t) for t in data['lemma_BERT']]

CPU times: total: 4min 42s
Wall time: 4min 42s


### Подготовка данных

#### TfidfVectorizer

In [12]:
features = lemma
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=12345, stratify=target)

In [13]:
print(np.array(features_train).shape, np.array(features_test).shape, target_train.shape, target_test.shape)

(143362,) (15930,) (143362,) (15930,)


#### BertModel

In [14]:
features_bert = lemma_bert
target_bert = data['toxic']

model = model_class.from_pretrained(pretrained_weights)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

max_len = 0
for i in features_bert:
    if len(i) > max_len:
        max_len = len(i)  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def bert_transform(df):
    padded = np.array([i + [0]*(max_len - len(i)) for i in df])
    attention_mask = np.where(padded != 0, 1, 0)

    batch_size = 100
    embeddings = []
    for i in notebook.tqdm(range(padded.shape[0] // batch_size +1)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        
    features = np.concatenate(embeddings)        
    return features

In [16]:
features_bert = bert_transform(features_bert)

  0%|          | 0/1593 [00:00<?, ?it/s]

In [17]:
features_bert_train, features_bert_test, target_bert_train, target_bert_test = train_test_split(
    features_bert, target_bert, test_size=0.1, random_state=12345, stratify=target_bert)

print(features_bert_train.shape, features_bert_test.shape, target_bert_train.shape, target_bert_test.shape)

(143362, 768) (15930, 768) (143362,) (15930,)


**Вывод**

Пропусков и дубликатов в данных не выявлено. Длину списка токенов ограничили 512 для возможности использовать предобученную модель `'bert-base-uncased'`. Для подготовки признаков было решено использовать параллельно 2 метода:
 - TfidfVectorizer
 - BERT

## Обучение

In [18]:
# очистка памяти

model = None
gc.collect()
torch.cuda.empty_cache()

# функция для подсчета времени

def exec_time(start, end):
    diff_time = end - start
    m, s = divmod(diff_time, 60)
    h, m = divmod(m, 60)
    s,m,h = int(round(s, 0)), int(round(m, 0)), int(round(h, 0))
    
    return "{0:02d}:{1:02d}:{2:02d}".format(h, m, s)

In [19]:
сlassifiers = [
                LogisticRegression(max_iter=1000),
                LGBMClassifier(random_state = 12345)
              ]

сlassifiers_dict = {0: 'LogisticRegression',
                    1: 'LGBM'}

param = {
             0:{'clf__C': range(5,16,1)},
             1:{'clf__learning_rate': [0.1, 0.25, 0.3, 0.5]},
                 
             2:{'C': range(5,16,1)},
             3:{'learning_rate': [0.1, 0.25, 0.3, 0.5]}
    
        }


best_F1 = 0
best_param = 0
best_model = ''
best_model_index = ''
index = 0
t = 0
bert_status = ''


def best(test=False):
    print('\nМОДЕЛЬ с лучшим значением F1 на валидации: {}'.format(best_model_index))
    print('BERT model: {}'.format(bert_status))
    print('F1 на валидации:  {:.4f}'.format(best_F1))
    if test == True:
            print('F1 на test:  {:.4f}'.format(F1))
    print('Параметры лучшей модели:', best_param)
    print("Общее время:", t)

  
f1 = make_scorer(f1_score)

count_tf_idf = TfidfVectorizer()

for reg in сlassifiers:
    for bert in ['false', 'true']:
        
        start = time.time()
        
        pipeline = Pipeline([
        ('vect', count_tf_idf),
        ('clf', reg)
                    ]) 
               
        if bert == 'false':     
            model = GridSearchCV(pipeline, param_grid=param[index], scoring=f1, cv=3) 
            model.fit(features_train, target_train)
        if bert == 'true':  
            model = GridSearchCV(pipeline[1], param_grid=param[index+2], scoring=f1, cv=3) 
            model.fit(features_bert_train, target_bert_train)
                           
        if model.best_score_ > best_F1:
            best_F1 = model.best_score_
            best_model_index = сlassifiers_dict[index]
            best_model = reg
            bert_status = bert
            best_param = model.best_params_ 
            end_1 = time.time() 
            t = exec_time(start,end_1)
        
            # сохранение модели
            final_model = 'finalized_model.sav'
            pickle.dump(model, open(final_model, 'wb'))
            
            with open('config.json', 'w') as f:
                json.dump(best_param, f)
            
        
        end = time.time()    
        print('\nМодель: {}'.format(сlassifiers_dict[index]))
        print('\nBERT model: {}'.format(bert))
        print('Лучшие параметры : {}'.format(model.best_params_))
        print('Лучшее значение F1 на валидации: {:.4f}'.format(model.best_score_))
        print("Общее время:", exec_time(start,end))
        print()
    index += 1
    
best()


Модель: LogisticRegression

BERT model: false
Лучшие параметры : {'clf__C': 13}
Лучшее значение F1 на валидации: 0.7791
Общее время: 00:06:19


Модель: LogisticRegression

BERT model: true
Лучшие параметры : {'C': 14}
Лучшее значение F1 на валидации: 0.7250
Общее время: 00:36:11


Модель: LGBM

BERT model: false
Лучшие параметры : {'clf__learning_rate': 0.3}
Лучшее значение F1 на валидации: 0.7760
Общее время: 00:04:06


Модель: LGBM

BERT model: true
Лучшие параметры : {'learning_rate': 0.25}
Лучшее значение F1 на валидации: 0.6769
Общее время: 00:03:38


МОДЕЛЬ с лучшим значением F1 на валидации: LogisticRegression
BERT model: false
F1 на валидации:  0.7791
Параметры лучшей модели: {'clf__C': 13}
Общее время: 00:06:19


**Вывод**

In [20]:
best()


МОДЕЛЬ с лучшим значением F1 на валидации: LogisticRegression
BERT model: false
F1 на валидации:  0.7791
Параметры лучшей модели: {'clf__C': 13}
Общее время: 00:06:19


## Тестирование

Проверим лучшую модель на test.

In [21]:
with open('config.json', 'r') as f:
    config = json.load(f)

model_1 = pickle.load(open(final_model, 'rb')) # загрузка модели
model_1.set_params(param_grid=config)

if bert_status == 'true':
    F1 = model_1.score(list(features_bert_test), list(target_bert_test))    

if bert_status == 'false':
    F1 = model_1.score(list(features_test), list(target_test))
    
print('F1 лучшей модели на test:   {:.4f}'.format(F1))

F1 лучшей модели на test:   0.7958


Проверим модель на адекватность, сравнив с исскуственной.

In [22]:
strategies = ['stratified', 'uniform']

for strategy in strategies:

    baseline_model = DummyClassifier(strategy=strategy)
    baseline_model.fit(features_train, target_train)

    predicted = baseline_model.predict(features_test)
    
    print('\nСтратегия модели:', strategy)
    print('F1 модели на test:  {:.4f}'.format(f1_score(target_test, predicted)))


Стратегия модели: stratified
F1 модели на test:  0.1097

Стратегия модели: uniform
F1 модели на test:  0.1653


Необходимый порог для F1 (0.75) достигнут. Модель прошла проверку на адекватность. 

**Вывод**

В проекте независимо друг от друга использовались 2 метода подготовки признаков для задачи NLP:
 - TfidfVectorizer
 - предобученная модель `bert-base-uncased`.

Стоит отметить, что второй метод потребовал значительно больше времени чем первый (около 1 часа работы на `gpu`) и показал себя на всех моделях хуже первого. 

В тесте участвовали следующие модели: `LogisticRegression`, `LGBM`. 

Лучший результат:

In [23]:
best(test=True)


МОДЕЛЬ с лучшим значением F1 на валидации: LogisticRegression
BERT model: false
F1 на валидации:  0.7791
F1 на test:  0.7958
Параметры лучшей модели: {'clf__C': 13}
Общее время: 00:06:19
